<a href="https://colab.research.google.com/github/katearb/test_case/blob/master/Data_exploring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание

Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 
Главные задачи, которые должен выполнять скрипт:
* Извлекать реплики с приветствием – где менеджер поздоровался. 
* Извлекать реплики, где менеджер представил себя. 
* Извлекать имя менеджера. 
* Извлекать название компании. 
* Извлекать реплики, где менеджер попрощался.

Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


In [1]:
import pandas as pd
from typing import List

# Первичный анализ

Первым делом познакомимся с данными и ответим на вопросы:
* Что из себя представляет датасет?
* Какие данные там содержатся?
* Что из себя представляет отдельная строка и как строки связаны между собой?

А также охарактеризуем тексты с различных сторон.

In [2]:
def get_dialogues(data: pd.DataFrame) -> List[str]:
  """
  Returns the dialogues in form of a string with lines and roles for futher printing
  """

  roles = data.groupby('dlg_id')['role'].apply(list)
  dialogues = data.groupby('dlg_id')['text'].apply(list)

  role_dialogues_joint = [[*zip(d_roles, texts)] 
                                    for d_roles, texts in zip(roles, dialogues)]

  role_dialogues_str = ['\n'.join([': '.join(line) for line in dialogue]) 
                                          for dialogue in role_dialogues_joint]

  return role_dialogues_str

In [3]:
# load the data
data = pd.read_csv('/content/drive/MyDrive/test_data.csv')

In [4]:
data.head(10)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
6,0,6,client,Да мы работаем с компанией которая нам подлива...
7,0,7,client,Как как бы уже до этого момента работаем все у...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
9,0,9,client,Как бы нет


In [69]:
print(get_dialogues(data)[0])

client: Алло
manager: Алло здравствуйте
client: Добрый день
manager: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
client: Ага
manager: Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
client: Да мы работаем с компанией которая нам подливает поэтому спасибо огромное
client: Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому
manager: Угу а на что вы обращаете внимание при выборе
client: Как бы нет
client: Да на выбор я уже не обращаю внимание я уже дома выбрал
manager: Что для вас приоритет
client: Поэтому типа много параметров
client: Выбор не выбор есть история сотрудничества которая более важна потому что вот смотрите вы же мне 1 раз звоните
client: А работаем мы там а уже давно а почему звоните только сейчас а не в течении своего времени когда работают
manager: Ну у вас срок заканчивается поэтому мы набрали + ко всему
client: Со

*(простите за лонгрид 😥)*

**Что стало известно от структуре датасета**:
* Датасет содержит реплики телефонных разговоров между менеджерами и клиентами. 
* Одна строка датафрейма содержит один небольшой относительно самостоятельный отрезок высказывания, однако является лишь частью целой реплики. Рискну преположить, деление производилось по фонетическому признаку, а именно по паузам в речи.

* Для каждой реплики представлена информация об авторе, о принадлежности к отельному диалогу и ее позиция в этом отдельном диалоге.

*  Для проверки соблюдения требований менеджером будет необходимо сгруппировать данные по диалогам.

**Что известно от репликах**

Текст -- затранскрибированные телефонные разговоры. При работе с таким родом текстов могут возникнуть следующие сложности:
1. Отсутствие заглавных букв в именах собственных и названиях компаний (а ведь это то, что нам и нужно найти:( ), а также знаков препинания.
2. Ошибки в распознавании речи
3. Сам факт устной речи со всеми ее особенностями: слова-паразиты, повторения, заикания, ошибки, частая несвязанность в высказывании.

**Общие замечания:**
1. Приветствия и прощания являются частью речевого этикета. Их особенность -- наличие устойчивых, стереотипных формулы в обращении. А значит, основываясь на анализе типичных формул приветствия и прощания среди менеджеров, можно выявить эти сами *паттерны* и по ним обнаружить в тексте необходимую информацию.
2. В моем представлении, "менеджер представляет себя" и "менеджер называет свое имя" -- одна и та же информация. Поэтому тэг "manager_self_represented" будет иметь флаг в тех репликах, где удалось обнаружить имя менеджера.

2. При извлечении имен также можно положиться на *паттерны* саморепрезентации ("Меня зовут X", "Я Y"), коллекцию имен и типичное расположение саморепрезентации в начале диалога. Аналогично для названий компаний.

3. Извлечение имен компаний -- достаточно сложная задача, а на русском языке до сих пор нет хорошего инструмента. Чтобы достичь наилучшего результата, для решения этой задачи я буду полагаться на 2 инструмента: парсинг по правилам и нахождение совпадений по набору известных компаний.

Для оценки соблюдения менеджером требований можно составить отдельный отчет по каждому диалогу.

**Возможные инструменты для решения задачи:**
1. Первый и самый очевидный: регулярные выражения. Выше я не раз говорила формулы и паттерны, для извелечения которых регулярки подходят отлично.
2. Можно создавать правила и воспользоваться парером Yargi.
3. Был бы рамеченный текст, можно было бы обучить какой-нибудь алгоритм для решения NER задачи, однако за неимением разметки, этот вариант отметается. Да и данных для обучения у нас не так много.
4. Синтаксическая и морфологическая разметка могла бы дать дополнительную информацию, однако она здесь кажется избыточной и неуместной с учетом специфики текстовой информации. Также мне кажется, что лексической информации должно хватить для выполнения задач.

Таким образом, система ругулярных выражений и ивристических правил должна здесь справиться. 

**Этапы работы:**
1. Проанализировать предложенные тексты, выявить паттерны для каждого типа интересующей информации.
2. Подключить к анализу лингвистическую экспертизу и собственные знания о языке, чтобы составить паттерны для регулярных выражений и правила, которые будут разнообразны и будут работать хорошо не только на этих текстах, но и на тех, которые мы не видели (То есть постараться не переобучиться и показать свою обобщающую способность на максимуме :) )
3. Построить систему для удобного использования.

Собственно, в этом ноутбуке будет находиться анализ, а имплементация работающего решения в отдельном .py файле.

Я заранее разделю данные на обучающую и тестовую выборки. В рамках анализа я буду ориенироваться только на первую часть, а на второй проверять, как хорошо справляется мое решение.
Конечно больно принимать такое решение, потому что данных у нас не так уж и много...но хочется хоть как-то провалидроваться.

# Train/test split

In [5]:
# choose the test samples
import random
random.seed(12)
test_ids = random.sample(range(2, 6), k=2)  # 0 and 1 are not included as I have already seen them

print('Test samples are', test_ids)

Test samples are [5, 3]


In [36]:
# define train and test data
data_train = data[~data['dlg_id'].isin(test_ids)]
data_test = data[data['dlg_id'].isin(test_ids)]

In [7]:
# check shape
data_train.shape, data_test.shape

((284, 4), (196, 4))

# Изучение текстов

In [432]:
# investigate the texts
for dlg in get_dialogues(data_train):
  print('======'*10)
  print(dlg)

client: Алло
manager: Алло здравствуйте
client: Добрый день
manager: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
client: Ага
manager: Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
client: Да мы работаем с компанией которая нам подливает поэтому спасибо огромное
client: Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому
manager: Угу а на что вы обращаете внимание при выборе
client: Как бы нет
client: Да на выбор я уже не обращаю внимание я уже дома выбрал
manager: Что для вас приоритет
client: Поэтому типа много параметров
client: Выбор не выбор есть история сотрудничества которая более важна потому что вот смотрите вы же мне 1 раз звоните
client: А работаем мы там а уже давно а почему звоните только сейчас а не в течении своего времени когда работают
manager: Ну у вас срок заканчивается поэтому мы набрали + ко всему
client: Со

1. Приветствия: 
  * 0-2. здравствуйте (отрезок 0)
  * 4 . -
2. Самопредставление и имя менеджера:
  * 0-2. Меня зовут ангелина (отрезок 1)
  * 4 . -

3. Название компании:
  * 0-2. [Меня зовут ангелина] компания диджитал бизнес (отрезок 1)
  * 4 . -

4. Прощание:
  1. Всего хорошего до свидания (отрезки -2, -1)
  2. До свидания (отрезок -1)
  3. -
  4. до свидания (отрезок -1)


Получилось не много. Видимо, в нашу выборку попала только Ангелина и еще какой-то анонимный менеджер. Время подключать свой опыт, справочники по речевому этикету и знания о языке.

# Подбор разнообразных шаблонов

* Еще возможные приветствия: добрый день, доброе утро, добрый вечер.

  Есть также неформальные варианты, которыми менеджер не должен пользоваться, однако перед нами стоит задача выявления наличия приветствия вне заивисмости от того, насколько оно корректно. Поэтому, можем рассмотреть и такие варианты: привет, приветствую, с добрым утром.

* Еще формулы представления себя менеджером: 
  * это [менеджер] X [компания|из компании], 
  * вам звонит X [компания|из компании], 
  * [компания Y] [менеджер] X,
  * меня зовут X я работаю в компании, 
  * вас беспокоит X [компания|из компании] 

  Менеджеры как правило представляются только по имени.

* Еще формулы представления компании: 
  * [приветствие] из компании Y, 
  * офис/подразделение Y, 
  * [приветствие] фирма/из фирмы Y, 
  * [приветствие] Y, 
  * из Y.

  Вообще, типов организаций очень много, и все они потенциально могут стоять здесь вместо слова "компания": стоматология, банк, магазин, студия, и многие-многие другие. Расширение этого списка зависит от того, какие встречаютя в данных. В наших данных нет никаких специфческих обозначений, поэтому оставим это для будущего улучшения парсера.

* Еще прощания: хорошего дня, всего доброго, хорошего вечера.

  Из неформальных: пока, до скорого.


Итак, мы выяснили типичные позиции тех или иных реплик в речи менеджера и паттерны для каждого типа информации. Следующий шаг, разработка шаблонов и создание скрипта. Это находится в файле DialoguesParser.py